# 8. Numerikus deriválás

*(Interpolációból, Taylor-sorfejtéssel, Richardson-extrapoláció, Rácsfüggvényeken ható mátrxalak)*

### P1. Feladat
Írjunk programot amely adott (egyenletes) $[x_0,x_1,..x_n]$ alappontokra és $[y_0,y_1,..y_n]$ függvényértékekre kiszámítja az $n$-edfokú interpolációból származó deriváltformula értékeit minden alapponton. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
xs = np.linspace(-1,2,10)
ys = np.exp(1/2*xs)

# osztott differenciás táblázat

def delta(n: int, m: int):
    assert n <= m
    
    if n == m:
        return ys[n]
    
    return (delta(n, m-1) - delta(n+1, m))/(xs[n] - xs[m])

def delta_table(xs: np.array, ys: np.array):
    assert len(xs) == len(ys)
    dX = xs.reshape((-1, 1)) - xs
    deltas = np.diag(ys) + 0.0
    
    for j in range(1, len(ys)):
        d = np.diag(deltas, j-1)
        deltas += np.diag((d[:-1] - d[1:])/np.diag(dX, j), j)
    
    return deltas

#Newton interpoláció deriváltja - az alappontokon ez lesz a derivált közelítése is

def poly_(xs,deltas):
    coeff=deltas[0,:]
    def g(z,k):
        v=0
        for j in np.arange(0,k):
            v+=np.prod(z - xs[0:k])/(z - xs[j]+1e-60)
        return v
    
    def h(z):
        v=0
        for j in np.arange(0,len(xs)):
            v+=g(z,j)*coeff[j]
        return v
    return h


In [ ]:
# interpolációs polinomok deriváltjának ábrázolása

h=3/200
xx=np.linspace(-1+h,2,200)

def deriv(xs,ys):
    deltas=delta_table(xs,ys)
    polyD=poly_(xs,deltas)
    polyD_vec=np.vectorize(polyD)
    dy=polyD_vec(xx) 
    return dy

plt.title('derivált közelítése')
for N in [2,3,4,8]:
    
    plt.plot(xx, deriv(np.linspace(-1,2,N),np.exp(1/2*np.linspace(-1,2,N))), label=f'N={N} pontos interpoláció')
plt.plot(xx, 1/2*np.exp(1/2*xx),label=f'1/2*e^(x/2)' )

plt.legend()
plt.show()

### P2. Feladat
Írjunk programot, amelyben $\sin(x)$ függvényt deriváljuk numerikusan. A $[0,2\pi]$ intervallumon egyenletes felosztás mellett alkalmazzuk a $f'(x_k)\approx\dfrac{f(x_k+h)-f(x_k-h)}{2h}$ közelítést minden rácsponton, ábrázoljuk is az eredményt. Nézzük meg mi történik, ha egymás után többször is végrehajtjuk ezt a műveletet.

In [ ]:
import matplotlib.pyplot as plt
n=100
h=2*np.pi/n
xs=np.linspace(0,2*np.pi-h,n)
ys = np.sin(xs)

def mtx_A(n: int) -> np.ndarray:
    A=(np.diag(-np.ones(n-1), -1)  + np.diag(np.ones(n-1), 1))
    A[0,-1]=-1
    A[-1,0]=1
    A/=(2*2*np.pi/(n))
    return A 



A=mtx_A(n)

plt.title('sin(x) deriváltjai')
for N in [1,2,3,4]:
    An=np.linalg.matrix_power(A,N)
    plt.plot(xs, An@ys, label=f'$A^{N}x$')

plt.legend()
plt.show()

### P3. Feladat

Tekinsük az alábbi egyenletet:
$$
x^5-10e^{-x} = 0
$$

Newton-iteráció segítségével keressük ennek az $[1,2]$ intervallumban lévő megoldását, úgy, hogy módosítjuk a Newton módszert a következőképpen: Az $x_{n+1}=x_n-\dfrac{f(x_n)}{f'(x_n)}$ iterációban $f'(x_n)$ helyett vegyük az $f'(x_n)\approx\dfrac{f(x_n)-f(x_{n-1})}{x_n-x_{n-1}}$ közelítést! Most 2 kezdőpontra is szükségünk lesz, ezek legyenek $1$ és $1,1$.

In [ ]:
import numpy as np

# Függvények definiálása
def f(x):
    return x**5-10*np.exp(-x)

# df(x,y):
    
# Newton-iteráció
def newton_iteration(x0,x1,max_iter=100, tol=1e-6):
    for i in range(max_iter):
        # Függvényértékek és deriváltak kiszámítása a jelenlegi pontban
        f_val0 = f(x0)
        f_val1 = f(x1)
        
        it_num=i
        
        
        df_val = (f_val1-f_val0)/(x1-x0)
        print(df_val)
        # Hiba kiszámítása
        error = np.abs(f_val1)
        
        # Ha elég közel vagyunk a megoldáshoz, kilépünk
        if error < tol:
            print("Megoldás találva:", x1)
            break
        
        # Newton lépés
        dx=-f_val1/df_val
        x0 = np.copy(x1)
        x1 += dx
        #print(x0)
        #print(x1)
        
    
    else:
        print("Nem sikerült megoldást találni")
        
    return x1, it_num

# Kezdeti pont beállítása
x0, x1 = 1, 1.1

# Newton-iteráció hívása
a,it = newton_iteration(x0, x1)

print("numerikus megoldás behelyettesítve:",f(a))
print("iterációk száma:", it)